# Running Classifiers 
**V.0.2 - Beta, [contributions](#contributions)**

The spam folder did not exist on email systems in the recent past. Emails that were relevant to the reader had to be manually (and painfully) sorted from an array of emails soliciting money or selling hoax products, among other things. The classification of our emails, by machines, into relevant emails and spam, has advanced to such a degree that we now take for granted that all the spam email we receive is automatically routed to the spam folder, needing little oversight from us. These machine classifiers use algorithms that are applicable to a wide variety of fields: written language; spoken language (e.g. "Hello Google", "Alexa", "Siri"); navigating driverless cars; and we'll also use them to understand brain activity. 

 This notebook, will walk you through the steps of labeling the fMRI signal, and then training and testing classifiers.


## Goal of this script
Using this script you will be able to use a classifier on your dataset. Additionally, you will be familiarized with the use of functions in your code. Specifically, we will accomplish the following:  
>1. Assign labels to every time-point (TR) in the dataset
>2. Time-shift the signal to be classified, taking into consideration the delayed hemodynamic response
>3. Collect BOLD data for all runs into one array
>4. Test out a classifier (SVM) on a group of subjects with a fixed set of parameters.
>5. Compute basic statistics to estimate means and confidence intervals  

### Pre-requisite
You should be familiar with the data loading and normalization steps in the 02-data-handling notebook.

Important terms you should be familiar with: TRs, labels.

## Table of Contents
[1. Loading and Normalization](#load_data)  
>[1.1 Label fMRI data](#label_data)  
>[1.2 Plot the different conditions](#plot_boxcar)  
>[1.3 Hemodynamic Lag: Time shift the labels](#label_shift)  
>[1.4 Load the fMRI data](#load_fmri)  

[2. Classification](#classification)  
>[2.1 Reshape data](#reshape)  
>[2.2 Model training](#model_training)  
>[2.3 Model testing](#model_testing)  
>[2.4 Test across participants](#across_ppts)  

Exercises
> [1](#ex1) [2](#ex2)




**Dataset** For this script we will use a localizer dataset from [Kim et al. (2017)](https://doi.org/10.1523/JNEUROSCI.3272-16.2017) again. Just to recap: The localizer consisted of 3 runs with 5 blocks of each category (faces, scenes and objects) per run. Each block was presented for 15s. Within a block, a stimulus was presented every 1.5s (1 TR). Between blocks, there was 15s (10 TRs) of fixation. Each run was 310 TRs. In the matlab stimulus file, the first row codes for the stimulus category for each trial (1 = Faces, 2 = Scenes, 3 = Objects). The 3rd row contains the time (in seconds, relative to the start of the run) when the stimulus was presented for each trial.

By this stage of analysis, you should have completed motion correction and any other required pre-processing on the fMRI data. In this example, all the data have been pre-processed for you.

**Python Indexing Reminder:** Everything begins at [0], not [1].


In [ ]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import nibabel as nib
import numpy as np
from nilearn.input_data import NiftiMasker
import scipy.io
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.svm import LinearSVC
from sklearn.model_selection import PredefinedSplit
from sklearn.preprocessing import StandardScaler
from brainiak.utils.fmrisim import _double_gamma_hrf as hrf_func
from brainiak.utils import fmrisim as sim


%matplotlib inline 
%autosave 5
sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})

In [ ]:
# load some helper functions
from utils import load_vdc_mask, load_vdc_epi_data, load_vdc_masked_data, label2TR
# load some constants
from utils import vdc_data_dir, vdc_all_ROIs, vdc_label_dict, vdc_n_runs, vdc_hrf_lag, vdc_TR, vdc_TRs_run

print('Here\'re some constants, which is specific for VDC data:')
print('data dir = %s' % (vdc_data_dir))
print('ROIs = %s' % (vdc_all_ROIs))
print('Labels = %s' % (vdc_label_dict))
print('number of runs = %s' % (vdc_n_runs))
print('1 TR = %.2f sec' % (vdc_TR))
print('HRF lag = %.2f sec' % (vdc_hrf_lag))
print('num TRs per run = %d' % (vdc_TRs_run))

<div class="alert alert-block alert-warning">
<strong> Note on data file paths:</strong> The data directory points to a specific location on the Adroit cluster, for the NEU480 class, at Princeton University, <strong> and no change to the path is required </strong>. If you are not in the NEU480 class or running on another cluster (e.g., scotty or spock) you will need to make a change to the data file path variable `vdc_data_dir` in `utils.py`.  
<br>

<strong>If you are not in the Princeton or Yale class</strong>, that uses this dataset, check the brainiak tutorial repository for information on where to download this dataset.
</div>

In [ ]:
# Directory structure of the VDC dataset
!head -50 {vdc_data_dir}README.txt

## 1. Loading and Normalization<a id="load_data"></a>

As per the procedure used in the notebook, 02-data-handling, load in the vdc dataset. To load in the data we are going to first run a series of commands that set the variable name and then store the data. The timing data are stored as a matrix by concatenating all runs.

Last time, you used a function called `load_vdc_stim_labels(subject_name)`, which import the labels for the fMRI data. The cell below shows source code for that function: 

In [ ]:
# Set the variables
sub = 'sub-01';

# Preset the variable size
stim_label = [];
stim_label_allruns = [];
for run in range(1, vdc_n_runs+1):
    # Specify the input variables
    in_file = '%s%s/ses-day2/design_matrix/%s_localizer_0%d.mat' % (vdc_data_dir, sub, sub, run)
    # Load in data from matlab
    stim_label = scipy.io.loadmat(in_file);
    stim_label = np.array(stim_label['data']);

    # Store the data
    if run == 1:
        stim_label_allruns = stim_label;
    else:       
        stim_label_allruns = np.hstack((stim_label_allruns, stim_label))

print("Loaded labels for", sub)

<div class="alert alert-block alert-warning">
<strong> Usage of hstack and vstack:</strong> hstack stacks arrays in sequence horizontally (column wise) and vstack stacks arrays in sequence vertically (row wise). More details can be found at 
[np.hstack](https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html) and [np.vstack](https://docs.scipy.org/doc/numpy/reference/generated/numpy.vstack.html).
</div>

<div class="alert alert-block alert-info">
<strong> Creating and using functions is an efficient way to program. </strong>  

Although we can load the data with the set of commands above, this isn't an efficient way to code if we want to re-use this code. A better way to code is to make a **function** that performs the loading of the data and then can be called with different input variables to load in new data. We do this below.

Functions are extremely useful for many reasons and so should be used everywhere: they allow you to remove redundancy in your code, reduce the likelihood of an error since if you update the function you update all of its uses, and you make your code much more readable.

A useful tutorial on functions is found [here](https://www.datacamp.com/community/tutorials/functions-python-tutorial).

One important thing to be aware of is how variables are shared between your workspace and a function. If you have variables in your workspace (i.e., any variables you have created in the usage of jupyter) then they will usually be accessible/usable in a function, regardless of whether they are used as input parameters. However, any variables you create in a function cannot be used in your workspace if you don't return them as outputs. For this reason it is easier if you keep the names of the variables in your function separate from the names in your workspace. This is turtles all the way down: if you have a function within a function then variables will be shared in the same way.

The function that we will now create is called `load_vdc_stim_labels`. You can see that this is a function since it starts with `def`.
<div>

In [ ]:
# Make a function for loading in the labels
def load_vdc_stim_labels(vdc_data_dir, subject_id):
    stim_label = [];
    stim_label_concatenated = [];
    for run in range(1, vdc_n_runs+1):
        in_file = '%s%s/ses-day2/design_matrix/%s_localizer_0%d.mat' % (
            vdc_data_dir, subject_id, subject_id, run)
        # Load in data from matlab
        stim_label = scipy.io.loadmat(in_file);
        stim_label = np.array(stim_label['data']);

        # Store the data
        if run == 1:
            stim_label_concatenated = stim_label;
        else:       
            stim_label_concatenated = np.hstack((stim_label_concatenated, stim_label))
    print("Loaded labels for", subject_id)
    return stim_label_concatenated

# Note: you won't see the printed "Loaded labels for..." until you run the function with inputs (see next step)

We now want to call this function and get its outputs.

In [ ]:
# Load in the labels of the data
stim_label_allruns = load_vdc_stim_labels(vdc_data_dir, sub)

### 1.1 Label fMRI data <a id="label_data"></a>

We need category labels to train a classifier for every timepoint in the fMRI data. Below, we assign these labels to every timepoint of fMRI data collected. These timepoints are also referred to as TRs.

This subject has 310 TRs but all others have 311. Some TRs will not have a label since there was no stimulus presented (e.g., the fixation periods between blocks). 

Recall that the third row of the timing file is the start time of a trial in seconds since the start of the run. We can thus convert each time stamp to a specific TR by taking that time stamp and dividing by the TR duration (1.5 s). 

In [ ]:
# Preset variables
_, events = stim_label_allruns.shape
events_run = int(events / vdc_n_runs)

# Preset the array with zeros
stim_label_TR = np.zeros((vdc_TRs_run * 3, 1))

# Cycle through the runs
for run in range(vdc_n_runs):

    # Cycle through each element in a run
    for i in range(events_run):

        # What element in the concatenated timing file are we accessing
        time_idx = run * (events_run) + i

        # What is the time stamp
        time = stim_label_allruns[2, time_idx]

        # What TR does this timepoint refer to?
        TR_idx = int(time / vdc_TR) + (run * (vdc_TRs_run - 1))

        # Add the condition label to this timepoint
        stim_label_TR[TR_idx] = stim_label_allruns[0, time_idx]

We have a function `label2TR` that will return a list of category label values for each TR.  It uses the following variables: `stim_labels`, `num_runs`, `TR`, and `TRs_run`.

In [ ]:
# Run function
stim_label_TR = label2TR(stim_label_allruns, vdc_n_runs, vdc_TR, vdc_TRs_run)

### 1.2 Plot the different conditions <a id="plot_boxcar"></a>

Below we create a box car plot (looks like square waves) of the timing of different conditions for the first run. 

In [ ]:
# Create a sequence of timepoints that a TR occurred on
tr_time = np.arange(0, (vdc_TRs_run - 1) * 1.5 + 1, 1.5)

# Plot the data
f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(tr_time, stim_label_TR[0:vdc_TRs_run, 0], c='orange')

ax.set_ylabel('Stimuli labels')
ax.set_xlabel('TR')

###  1.3 Hemodynamic Lag: Time shift the labels <a id="label_shift"></a>

The brain response that we measure with fMRI is slow which means there is a lag between when events occur and when we expect to find changes in the BOLD signal. Below we plot the expected neural response to a single event.
More details about hemodynamic response can be found [here].(https://en.wikipedia.org/wiki/Haemodynamic_response)

In [ ]:
# Help on hrf_func
help(hrf_func)

hrf = hrf_func(temporal_resolution=1)

# Plot the canonical double gamma HRF
f, ax = plt.subplots(1,1, figsize = (10, 5))
ax.plot(range(30), hrf)

ax.set_title("Hemodynamic Response Function (HRF)")
ax.set_xlabel('Time in secs')

**1.3.1 Modelling the HRF**  

We now model the HRF for two different scenarios: a sequence of stimuli presented close together (block design), and individual stimuli presented far apart (event design). 

**1.3.1.a. What does the expected neural response look like if events are shown continuously for 10 seconds?**

The various parameters used are described below: 
> `trDuration`:how long is each TR.  
> `numTRs`: total number of TRs in the experiment.  
> `onsets`: timepoint in experiment when the stimulus is turned on.  
>`event_duration`: how long is each event. This could be the duration of block (for block design experiments) or a trial (for event-related designs).  

In [ ]:
# Specify the volume parameters
trDuration = 2  # seconds
numTRs = 40 # How many TRs will you generate?
total_time = int(numTRs * trDuration)
temporal_res = 0.5 #1/trDuration

In [ ]:
#The event onsets at 20 seconds.
# The block lasts for 10 seonds (5 TRs).

stim_A = sim.generate_stimfunction(onsets=[20], 
                                                   event_durations=[10], 
                                                   total_time=total_time,
                                                   temporal_resolution=temporal_res 
                                                   )

In [ ]:
plt.plot(stim_A)
plt.title('Stimulus Timing: Block 10sec (5 TRs)')
plt.xlabel('TRs')

In [ ]:
signal_func_A = sim.convolve_hrf(stimfunction=stim_A,
                                   tr_duration=trDuration,
                                   temporal_resolution=temporal_res,
                                   scale_function=1,
                                   )

In [ ]:
plt.plot(signal_func_A)
plt.title('HRF: Block 10sec (5 TRs)')
plt.xlabel('TRs')

**1.3.1.b. Next we examine the expected neural response look like if two events evoke the same response, 10 seconds apart?**

In [ ]:
#The events onset at 20 and 30 seconds.
# Each event lasts for 2 seconds (1 TR).
stim_B = sim.generate_stimfunction(onsets=[20,30], 
                                                   event_durations=[2,2], 
                                                   total_time=total_time,
                                                   temporal_resolution=temporal_res 
                                                   )

In [ ]:
plt.plot(stim_B)
plt.title('Stimulus Timing: Two events 10sec(5 TRs) apart')
plt.xlabel('TRs')

In [ ]:
signal_func_B = sim.convolve_hrf(stimfunction=stim_B,
                                   tr_duration=2,
                                   temporal_resolution=temporal_res,
                                   scale_function=1,
                                   )

In [ ]:
plt.plot(signal_func_B)
plt.title('HRF: Two events 10sec (5 TRs) apart')
plt.xlabel('TRs')

**Observation**  
Note the difference in the plots for the block and the two event signals. When stimuli are presented in blocks, the HRF sustains itself for the duration of the block. For the two event stimuli, presented far apart, the hemodynamic signal increases and falls after the first event and then reovers after the second stimuli is shown, before finally going down to baseline. 

**Exercise 1:** <a id="ex1"></a>From what you just learned about the HRF, why might an experimenter want to keep stimuli separated in time?

**1.3.2 Time Shift the VDC Labels**  
To account for the hemodynamic lag in the neural response, we can shift the timecourse of events. First let's plot this timecourse.

In [ ]:
n_conditions = len(vdc_label_dict)
cur_pals = sns.color_palette('colorblind', n_colors=n_conditions)

# Create a sequence of timepoints that a TR occurred on
tr_time = np.arange(0, (vdc_TRs_run - 1) * 1.5 + 1, 1.5)
time_vals = stim_label_allruns[2, 0:150]
labels = stim_label_allruns[0, 0:150]

f, ax = plt.subplots(1,1, figsize = (14, 5))
    
# plot the label for each condition
for i_cond in range(n_conditions): 
    label = list(vdc_label_dict.keys())[i_cond]
    temp_mask = label == labels
    ax.scatter(time_vals[temp_mask], labels[temp_mask], 
               color = cur_pals[i_cond], marker = 'o')
ax.legend(vdc_label_dict.values())
    
# plot the stimuli as a line 
# ax.plot(time_vals, labels, color = 'black', alpha = .5)
ax.plot(tr_time, stim_label_TR[0:vdc_TRs_run, 0], c='orange', alpha = .5)

ax.set_yticks(list(vdc_label_dict.keys()))
ax.set_yticklabels(vdc_label_dict.values())

ax.set_title('Stimulus Presentation for Run 1')
ax.set_xlabel('Time (seconds)')


We need to incorporate this time-shift when we extract the BOLD signal for classification. One way to accomplish this is to shift the labels by 4-5 seconds and extract the BOLD signal for the non-zero labels. As one TR = 1.5 seconds, we'll shift by 3 TRs.

In [ ]:
# Shift the data a certain amount
print(vdc_hrf_lag) # In seconds what is the lag between a stimulus onset and the peak bold response
shift_size = int(vdc_hrf_lag / vdc_TR)  # Convert the shift into TRs

# Create a function to shift the size
def shift_timing(label_TR, TR_shift_size):
    
    # Create a short vector of extra zeros
    zero_shift = np.zeros((TR_shift_size, 1))

    # Zero pad the column from the top
    label_TR_shifted = np.vstack((zero_shift, label_TR))

    # Don't include the last rows that have been shifted out of the time line
    label_TR_shifted = label_TR_shifted[0:label_TR.shape[0],0]
    
    return label_TR_shifted

# Apply the function
stim_label_TR_shifted = shift_timing(stim_label_TR, shift_size)

In [ ]:
# Add the boxcar to the plot

f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(tr_time, stim_label_TR_shifted[0:310], c='orange')

ax.set_ylabel('Stimuli labels')
ax.set_xlabel('TR')

Note how the shifted boxcar is not plotted automatically into the cell above. This is because plotting goes into the figure after plt.figure() was called and will keep writing there (automatically sets 'hold on' if you are familiar with matlab) until you call a new plt.figure().

### 1.4 Load the fMRI data<a id="load_fmri"></a>

As in the exercise from week 1 we will load in, mask, and z-score the data.

In [ ]:
print('available ROIs: ', vdc_all_ROIs)

In [ ]:
roi_name = 'FFA'

# Apply the function to pull out the mask data
epi_mask_data_all = load_vdc_masked_data(vdc_data_dir, sub, vdc_all_ROIs)

# Check the dimensionality of the data
print('voxel by TR matrix - shape: ', epi_mask_data_all[vdc_all_ROIs.index(roi_name)].shape)
print('label list - shape: ', stim_label_TR_shifted.shape)

## 2. Classification<a id="classification"></a>

We will now build a very basic classifier to categorize our data. 

In the example of email, spam mail has certain characteristics (or features) such as: sent from an unknown sender; sells products; asks for money deposits. These features are converted into mathematical vectors in a language feature space and computer programs are trained on these feature vectors,  _from known examples of relevant and spam emails_, to categorize emails as either relevant or spam. For brain activity measured by fMRI, the voxel signals serve as the features. If we were showing pictures of faces, scenes, and objects, and collecting fMRI data while the subject was viewing the pictures, we know what picture was shown at each time-point. The stimulus type at each timepoint serves as the stimulus label. The signal from voxels at each timepoint correspond to the features of that picture. From this known set of stimulus labels and features, we can train a classifier to distinguish between pictures of faces, scenes, and objects.

Once the classifier training is accomplished, we still do not know how this classifier will perform when it has to make a prediction on fMRI signals for an image that it was not trained on, e.g. a new picture of a face. To determine if the classifier can predict the stimulus label of an _unseen stimulus_, we test the classifier we created, on an _unseen dataset_, and determine its prediction accuracy. If the classifier makes predictions above chance (random guessing will lead to a 33.33% accuracy for 3 categories), the classifier has "mind read" the category of the stimulus.

We will try the SVM classifier in this exercise.  
<div class="alert alert-block alert-info">
Each classifier has a number of parameters that can be changed to affect the sensitivity of the classification (called hyper-parameters). For now, we will hard code these parameters. We will cover more principled ways to do classification in a future notebook.
<div>

### 2.1 Reshape data <a id="reshape"></a>

First, we extract the time points for which we have stimulus labels (only Faces, Places, and Objects)-- i.e., we drop the time-points from the BOLD signal that refer to the fixation periods.

In [ ]:
# Extract bold data for non-zero labels
def reshape_data(label_TR_shifted, masked_data_all):
    label_index = np.nonzero(label_TR_shifted)
    label_index = np.squeeze(label_index)
    # Pull out the indexes
    indexed_data = np.transpose(masked_data_all[:,label_index])
    nonzero_labels = label_TR_shifted[label_index] 
    return indexed_data, nonzero_labels

bold_data, labels = reshape_data(
    stim_label_TR_shifted, epi_mask_data_all[vdc_all_ROIs.index(roi_name)]
)

# What is the dimensionality of the data? We need the first dim to be the same
print(bold_data.shape)
print(labels.size)

### 2.2 Leave-One-Run-Out (LORO) training and testing  <a id="model_training"></a>

We have our vdc dataset to train our classifier. From where do we get an unseen dataset to test the accuracy of the classifier? We use LORO. A typical fMRI experiment has multiple runs, wherein the same categories are shown in all runs. One trick we can use is to create an unseen dataset is to not train the classifier on all runs, let's say we Leave One Run Out from the training set. This left out run is now our unseen dataset -- the classifier was never trained on this data. We can use this dataset to test the accuracy of the classifier.

Furthermore, we can loop through the runs, leaving out a run each time (also known as folding), and test multiple times. Let's say we have 3 runs: 1,2,3. Then, we can establish the sequence of training and testing, giving us 3 independent tests of the classifier accuracy, as shown in the table below.

| Fold | Train | Test |
| --- | --- | --- |
|1 |  run1, run2 | run3 |
|2 |  run2, run3 | run1 |
|3 |  run3, run1 | run2 |
  

**Using LORO ensures that the classifier is tested on an unseen, independent dataset.**

#### 2.2.1. Manually create a left-out run.

In [ ]:
# get run ids (works similarity to cv_ids)
run_ids = stim_label_allruns[5,:] - 1 

# select a run
holdout_run_ids = 0
# make an index list with one run left out.
train_runs = run_ids != holdout_run_ids

In [ ]:
# let's look at what the runs that will be used for training (value=1) and runs that 
# will be used for testing (value=0).
plt.plot(train_runs)
plt.title('train_runs')
plt.ylabel('value')
plt.xlabel('index')

**2.2.2 Normalization**

This is an important step in classification (see notebook 02-data-handling). Normalization puts feature values in a similar range of values. This reduces variance across features and prevents classifiers from being biased towards features with large values.


In [ ]:
def normalize(bold_data_, run_ids):
    """normalized the data within each run
    
    Parameters
    --------------
    bold_data_: np.array, n_stimuli x n_voxels
    run_ids: np.array or a list
    
    Return
    --------------
    normalized_data
    """
    scaler = StandardScaler()
    data = []
    for r in range(vdc_n_runs):
        data.append(scaler.fit_transform(bold_data_[run_ids == r, :]))
    normalized_data = np.vstack(data)
    return normalized_data

In [ ]:
bold_data_normalized = normalize(bold_data, run_ids)

# split the training set and test set ... 
X_train = bold_data_normalized[train_runs,]
y_train = labels[train_runs]
X_test = bold_data_normalized[np.logical_not(train_runs),]
y_test = labels[np.logical_not(train_runs)]

### 2.3. Classifiers

We can use a variety of classifiers for our analysis. They are easily accessed by calling scikit-learn libraries. In the example below, we use a linear support vector machine. It is one of the most commonly used classifiers in cognitive neuroscience as it is quite to robust to low number of training samples and outliers in the data.

You can also explore a variety of classifiers here: http://scikit-learn.org/stable/supervised_learning.html

#### 2.3.1. Build a classifier and test on the held out run.

The `X_train` data was created by excluding one run. We use this to train a classifier and then test it on `X_test`, the dataset that was created from the held out run.

In [ ]:
# fit a classifier model on the training set 
model = LinearSVC(C=1)
# is hardcoded. Optimizing these parameters will be covered in later notebooks.
model.fit(X_train, y_train)

# compute your evaluation on the test set
score = model.score(X_test, y_test)
print('Accuracy = %s' % score)

#### 2.3.2. Build a classifier by training and testing on multiple folds.

We use scikit learn libraries to automate the task of creating multiple folds. The `PredefinedSplit` method does this automatically.

<div class="alert alert-block alert-warning">
<strong> The test dataset must always be independent of the train dataset.</strong>
<br>
No matter which method is adopted, you should always make sure that the train and test datasets are independent. It's best to exercise caution and make sure any folding method that you use is working the way you want it to. You could do this be getting the rowids of the training and test datasets and making sure that they have no overlap and belong to unique runs.
<div>


As an example, if you do not separate the training and test dataset, you will get inflated accuracies. We will cover more of this aspect in future notebooks.

** This is what you should not be doing.**

In [ ]:
"""double dipping"""

# data normalizer 
scaler = StandardScaler()
# classifier 
model = LinearSVC() 

X_train = scaler.fit_transform(bold_data)
# fit a svm
model.fit(X_train, labels)
# calculate the accuracy for the hold out run
score = model.score(X_train, labels)
print(score)

** Now we split the dataset into training and testing in the correct way.**

#### handle the train-test split with sklearn
Here's the documentation of `PredefinedSplit`: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.PredefinedSplit.html


In [ ]:
"""demo"""
demo_run_ids = [1, 1, 0, 0]
ps = PredefinedSplit(demo_run_ids)
for train_index, test_index in ps.split():
    print("train ids:", train_index, "test ids:", test_index)

In [ ]:
# loop over all runs 
scores = []

# split data according to run ids 
ps = PredefinedSplit(run_ids)

# classifier 
model = LinearSVC() 
for train_index, test_index in ps.split():
    X_train, X_test = bold_data_normalized[train_index], bold_data_normalized[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    # fit a svm
    model.fit(X_train, y_train)
    # calculate the accuracy for the hold out run
    score = model.score(X_test, y_test)
    scores.append(score)
print(scores)

**2.3.3 Classifier Performance**  
To determine how good is the accuracy of a classifier, we can compare it to random guessing. In the VDC dataset, we have 3 equally represnted sets of stimuli, and thus random guessing will result in being correct 1/3 of the time.

### 2.4 Perform the analysis across participants <a id="across_ppts"></a>

The next step is to run a classifier on a group of subjects. We will now create the condition variable, stim_label_TR, for all subjects and then use it to pull out the relevant participant data and then feed it into a classifier.

Before you run this command, make sure you save your notebook. You will be loading in a lot of data and so it might run into memory issues and crash your job. If you do have issues, change how many cores/memory you are using when you launch this notebook.

The code below loops through subjects and calls a classifier. Use this function for the question below.

In [ ]:
# pack the classification code as a function 
def decode(X, y, cv_ids, model): 
    """
    Parameters
    --------------
    X: np.array, n_stimuli x n_voxels
    y: np.array, n_stimuli, 
    cv_ids: np.array - n_stimuli, 
    
    Return
    --------------
    models: a list of models 
    scores: a list of scores
    """
    scores = []
    models = []
    ps = PredefinedSplit(cv_ids)
    for train_index, test_index in ps.split():
        # enter your code below... 
        
        # split the data  
        
        # train the decoder
        
        # score it on a hold-out set
    
        # save stuff 
        models.append(model)
        scores.append(score)
    return models, scores

In [ ]:
# run the function 
model = LinearSVC()  
models, scores = decode(X=bold_data_normalized, y=labels, cv_ids=run_ids, model=model)
print('Decoding accuracy across the 3 runs: ', scores)

**Exercise 2:** <a id="ex2"></a> What is the average accuracy across all participants of SVM?

## Contributions <a id="contributions"></a> 

M. Kumar, C. Ellis and N. Turk-Browne produced the initial notebook  
T. Meissner minor edits  
Q. Lu a lot of edits... 